# Introduction

Pembuatan Algoritma Prediksi Stok Inventori (inventory prediction)
Kembangkan sebuah algoritma prediksi stok inventori yang dapat memberikan saran mengenai jumlah item yang perlu disiapkan di setiap negara.

In [48]:
#modul persiapan dan penampilan dataset
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

# modul dari data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from feature_engine.outliers import Winsorizer , OutlierTrimmer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#untuk model liner regresi
from sklearn.linear_model import LinearRegression

#untuk model evaluaasi
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_percentage_error, mean_absolute_error

# Data Loading

In [49]:
#data loading 
df = pd.read_csv('fact_orders.csv')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,1/12/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1/12/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,9/12/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,9/12/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,9/12/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,9/12/2011 12:50,4.15,12680.0,France


In [50]:
#melihat data yang ada di kolom dan tipenya
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [51]:
#mlihat apakah ada missing value
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [52]:
#memngisi mising value dengan menggunakan modus
most_frequent_value = df['Description'].mode()[0]
df['Description'] = df['Description'].fillna(most_frequent_value)

In [53]:
#melihat missing value
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [54]:
#membuat function agar kategori produk tidak terlalu banyak
def kategori_berdasarkan_awalan(description):
    if description.startswith(('A', 'B')):
        return 'Product A-B'
    elif description.startswith(('C', 'D')):
        return 'Product C-D'
    elif description.startswith(('E', 'F')):    
        return 'Product E-F'
    elif description.startswith(('G', 'H')):
        return 'Product G-H'
    elif description.startswith(('I', 'J')):
        return 'Product I-J'
    elif description.startswith(('K', 'L')):
        return 'Product K-L'
    elif description.startswith(('M', 'N')):
        return 'Product M-N'
    elif description.startswith(('O', 'P')):
        return 'Product O-P'
    elif description.startswith(('Q', 'R')):
        return 'Product Q-R'
    elif description.startswith(('S', 'T')):
        return 'Product S-T'
    elif description.startswith(('U', 'V')):
        return 'Product U-V'
    elif description.startswith(('W', 'X')):
        return 'Product W-X'
    elif description.startswith(('Y', 'Z')):
        return 'Product Y-Z'

In [55]:
#menjalankan function
df['Description'] = df['Description'].apply(kategori_berdasarkan_awalan)

In [56]:
#mengecek pakah masih ada missing value
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description     18231
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [57]:
#mengisi data missing value dengan modus
most_frequent_values = df['Description'].mode()[0]
df['Description'] = df['Description'].fillna(most_frequent_values)

In [58]:
#missing value di deskriprion sudah hilang
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

# Feature Selection and Feature Engineering

In [59]:
#membuat variable data baru
df1=df[['Description','Quantity','UnitPrice','Country']]

In [60]:
#membagi data yang sudah di future selection menjadi x dan y dengan y yaitu kolom target 
X= df1.drop('Quantity', axis=1) #fitur
y= df1['Quantity'] #target

In [61]:
#fungsi membagi tabel menjadi xtrain,xtest,ytrain,dan ytest dimana nantinya akan di evaluasi setelah data dicleaning. 
#text size berada pada 0.2 menandakan bahwa 20% data akan di test dan 80% ada akan di train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

print('Train Size : ', X_train.shape)
print('Test Size  : ', X_test.shape)

Train Size :  (433527, 3)
Test Size  :  (108382, 3)


In [62]:
#fungsi memilih kolom pada xtrain dan xtest yang sudah ditest pada fungsi kolerasi di atas
X_train=X_train[['Description','UnitPrice','Country']]
X_test=X_test[['Description','UnitPrice','Country']]

In [63]:
#membagi xtrain menjadi kolom numerical dan kategorical untuk nantinya dilatih dan ditest
train_num_columns = X_train['UnitPrice']
train_cat_columns = X_train[['Description','Country']]
print('Numerical Columns : ', train_num_columns)
print('Categorical Columns : ', train_cat_columns)

Numerical Columns :  453248    0.83
39751     1.65
375256    1.65
451651    1.06
91578     6.75
          ... 
215699    2.55
535549    2.46
366848    0.83
452227    2.10
71530     2.55
Name: UnitPrice, Length: 433527, dtype: float64
Categorical Columns :          Description         Country
453248  Product S-T  United Kingdom
39751   Product I-J  United Kingdom
375256  Product K-L  United Kingdom
451651  Product Q-R  United Kingdom
91578   Product C-D  United Kingdom
...             ...             ...
215699  Product G-H  United Kingdom
535549  Product S-T  United Kingdom
366848  Product S-T  United Kingdom
452227  Product G-H  United Kingdom
71530   Product A-B  United Kingdom

[433527 rows x 2 columns]


In [64]:
#membagi xtest menjadi kolom numerical dan kategorical untuk nantinya dilatih dan ditest
test_num_columns = X_test['UnitPrice']
test_cat_columns = X_test[['Description','Country']]
print('Numerical Columns : ', test_num_columns)
print('Categorical Columns : ', test_cat_columns)

Numerical Columns :  132701    2.95
451617    8.95
313583    2.46
84285     1.25
475344    1.45
          ... 
190827    0.42
451439    0.29
154238    0.42
174865    0.42
326453    2.95
Name: UnitPrice, Length: 108382, dtype: float64
Categorical Columns :          Description         Country
132701  Product O-P  United Kingdom
451617  Product Q-R  United Kingdom
313583  Product K-L  United Kingdom
84285   Product S-T  United Kingdom
475344  Product G-H  United Kingdom
...             ...             ...
190827  Product W-X  United Kingdom
451439  Product W-X  United Kingdom
154238  Product U-V  United Kingdom
174865  Product S-T  United Kingdom
326453  Product Q-R  United Kingdom

[108382 rows x 2 columns]


In [65]:
#pada xtrain dan xtest yang sudah dibagi buat dalam bentuk dataframe agar nantinya bisa diproses

X_train_num = pd.DataFrame(train_num_columns)
X_train_cat = pd.DataFrame(train_cat_columns)

X_test_num = pd.DataFrame(test_num_columns)
X_test_cat = pd.DataFrame(test_cat_columns)

In [66]:
#menghitung persebaran jenis data dengan metode skewness
print(f"Skewness dari distance :{X_train_num['UnitPrice'].skew()}")

Skewness dari distance :187.58490283779963


In [67]:
#menghapus outlier pada kolom limit_balance di X train dan X test dengan tukey karena x test skewnessnya lebih dari 1 dan x train skewnessnya mendekati 1 dan outlier kurangd air 5%
Trimmer_price = OutlierTrimmer(capping_method='iqr',
                             tail='both',
                             fold=1.5,
                             variables=['UnitPrice'])

X_train_trimmer = Trimmer_price.fit_transform(X_train)
X_test_trimmer = Trimmer_price.transform(X_test)

In [68]:
train_num_columns

453248    0.83
39751     1.65
375256    1.65
451651    1.06
91578     6.75
          ... 
215699    2.55
535549    2.46
366848    0.83
452227    2.10
71530     2.55
Name: UnitPrice, Length: 433527, dtype: float64

In [69]:
#menggunakan standard scaler karena data normal pada x train
scaler = StandardScaler()
model = scaler.fit(X_train_num)
train_scaller = model.transform(X_train_num)

In [70]:
#hasil dari data yang sudah di scalling
train_scaller

array([[-0.0372399 ],
       [-0.0292302 ],
       [-0.0292302 ],
       ...,
       [-0.0372399 ],
       [-0.02483464],
       [-0.02043908]])

In [71]:
#melakukan scaling pada kolom numerikal di x test
test_Scaller = scaler.transform(X_test_num)

In [72]:
##hasil dari data yang sudah di scalling
test_Scaller

array([[-0.01653191],
       [ 0.04207559],
       [-0.02131819],
       ...,
       [-0.04124474],
       [-0.04124474],
       [-0.01653191]])

In [73]:
#membuat variabel baru untuk kolom kategorical untuk di encoding
X_train_nominal = X_train[['Description','Country']]
X_test_nominal = X_test[['Description','Country']]

In [74]:
# ubah dalam bentuk dataframe
X_train_nominal = pd.DataFrame(X_train_nominal)
X_test_nominal = pd.DataFrame(X_test_nominal)
X_train_nominal

,Description,Country
453248,Product S-T,United Kingdom
39751,Product I-J,United Kingdom
375256,Product K-L,United Kingdom
451651,Product Q-R,United Kingdom
91578,Product C-D,United Kingdom
...,...,...
215699,Product G-H,United Kingdom
535549,Product S-T,United Kingdom
366848,Product S-T,United Kingdom
452227,Product G-H,United Kingdom


In [75]:
#melakukan encoding pada data x train dan x test kategorical dengan variabel hanya nama saja karena surge multiplier sudah berbentuk numeric
encoder= OneHotEncoder(sparse=False) #array

encoder.fit(X_train_nominal)
X_train_ohe= encoder.transform(X_train_nominal)
X_test_ohe= encoder.transform(X_test_nominal)


c:\Users\kenne\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [76]:
#hasil dari x train kategorical yang sudah di encoding
X_train_ohe

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.]])

In [77]:
#hasil dari x test kategorical yang sudah di encoding
X_test_ohe

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [78]:
#menampilkan jenis data yang ada di kolom nama kategorical
encoder.get_feature_names_out()

array(['Description_Product A-B', 'Description_Product C-D',
       'Description_Product E-F', 'Description_Product G-H',
       'Description_Product I-J', 'Description_Product K-L',
       'Description_Product M-N', 'Description_Product O-P',
       'Description_Product Q-R', 'Description_Product S-T',
       'Description_Product U-V', 'Description_Product W-X',
       'Description_Product Y-Z', 'Country_Australia', 'Country_Austria',
       'Country_Bahrain', 'Country_Belgium', 'Country_Brazil',
       'Country_Canada', 'Country_Channel Islands', 'Country_Cyprus',
       'Country_Czech Republic', 'Country_Denmark', 'Country_EIRE',
       'Country_European Community', 'Country_Finland', 'Country_France',
       'Country_Germany', 'Country_Greece', 'Country_Hong Kong',
       'Country_Iceland', 'Country_Israel', 'Country_Italy',
       'Country_Japan', 'Country_Lebanon', 'Country_Lithuania',
       'Country_Malta', 'Country_Netherlands', 'Country_Norway',
       'Country_Poland', 'Coun

In [79]:
#memanggil nama pada x train ohes dan x test ohes agar ada namanya 
x_train_ohes=pd.DataFrame(X_train_ohe, columns=encoder.get_feature_names_out())
x_test_ohes=pd.DataFrame(X_test_ohe, columns=encoder.get_feature_names_out())

In [82]:
#reset index kolom numerical x train dan x text
X_train_num_fixs = X_train_trimmer['UnitPrice'].reset_index(drop=True)
X_test_num_fixs = X_test_trimmer['UnitPrice'].reset_index(drop=True)

In [83]:
#menyamakan index dengan fungsi iloc
X_train_ohes_new = x_train_ohes.iloc[X_train_num_fixs.index]
X_test_ohes_new = x_test_ohes.iloc[X_test_num_fixs.index]

In [84]:
#melakukan concat
X_train_final = pd.concat([X_train_num_fixs, X_train_ohes_new], axis=1)
X_test_final = pd.concat([X_test_num_fixs, X_test_ohes_new], axis=1)

In [85]:
#menyamakan index y train dengan iloc
y_train = y_train.iloc[X_train_num_fixs.index]
y_test = y_test.iloc[X_test_num_fixs.index]

In [86]:
#membuat data frame
y_train=pd.DataFrame(y_train)
y_train

,Quantity
453248,3
39751,100
375256,6
451651,72
91578,2
...,...
532172,2
251098,4
78127,1
42624,4


# Model Definition

In [87]:
#membuat variabel baru penampung linear regresi
model_lin_reg = LinearRegression()

In [88]:
#memanggil model
model_lin_reg

LinearRegression()

# Model Training

In [89]:
#melakukan model training
model_lin_reg.fit(X_train_final, y_train)

LinearRegression()

In [90]:
#melakukan model training
model_lin_reg.fit(X_train_final, y_train)

LinearRegression()

In [91]:
#melaukan prediksi pada y train dan y test
y_train_pred = model_lin_reg.predict(X_train_final)
y_test_pred = model_lin_reg.predict(X_test_final)

print(y_train_pred)

print('')

print(y_test_pred)

print("\nJumlah baris y_train:", y_train.shape[0])
print("Jumlah baris y_test:", y_test.shape[0])
print("Jumlah baris y_train_pred:", len(y_train_pred))
print("Jumlah baris y_test_pred:", len(y_test_pred))

[[ 7.52612305]
 [11.34106445]
 [ 8.1472168 ]
 ...
 [ 7.52612305]
 [13.48217773]
 [ 9.44848633]]

[[10.54321289]
 [ 7.3371582 ]
 [ 8.15063477]
 ...
 [84.37158203]
 [ 8.15771484]
 [11.32983398]]

Jumlah baris y_train: 401796
Jumlah baris y_test: 100486
Jumlah baris y_train_pred: 401796
Jumlah baris y_test_pred: 100486


# Model Evaluation

In [92]:
# MAE (Mean Absolute Error) Train and Test:
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
print(f'MAE Train:', mae_train)
print(f'MAE Test:', mae_test)

# MSE (Mean Squared Error) Train and Test:
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
print(f'\nMSE Train:', mse_train)
print(f'MSE Test:', mse_test)

# RMSE (Root Mean Squared Error) Train and Test:

rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)
print(f'\nRMSE Train:', rmse_train)
print(f'RMSE Test:', rmse_test)

# R2 Score (Coefficient of Determination) Train and Test:

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
print(f'\nR2 Train:', r2_train)
print(f'R2 Test:', r2_test)

# MAPE (Mean Absolute Percentage Error) Train and Test:
mape_train_rounded = mean_absolute_percentage_error(y_train, y_train_pred)* 100
mape_test_rounded = mean_absolute_percentage_error(y_test, y_test_pred) * 100

print('\nMAPE Train:', mape_train_rounded)
print('MAPE Test:', mape_test_rounded)
print('')
print(f'mean :{y.mean()}')

MAE Train: 11.29812737913701
MAE Test: 10.967368846196548

MSE Train: 48753.71744976684
MSE Test: 4048.0688039401693

RMSE Train: 220.80243986370903
RMSE Test: 63.62443558838199

R2 Train: 0.0008788526691486664
R2 Test: 0.011234848953199483

MAPE Train: 325.203503717146
MAPE Test: 325.3528430121015

mean :9.55224954743324


# Kesimpulan

MAE (Mean Absolute Error): MAE pada data pelatihan adalah sekitar 11.30 dan pada data pengujian adalah sekitar 10.97. Ini menunjukkan bahwa model memiliki kinerja serupa pada data pelatihan dan pengujian, dengan perbedaan yang relatif kecil antara kedua metrik ini.

MSE (Mean Squared Error): MSE pada data pelatihan adalah sekitar 48753.72 dan pada data pengujian adalah sekitar 4048.07. Ini menunjukkan bahwa kesalahan model cenderung lebih besar pada data pelatihan dibandingkan dengan data pengujian.

RMSE (Root Mean Squared Error): RMSE pada data pelatihan adalah sekitar 220.80 dan pada data pengujian adalah sekitar 63.62. Ini menunjukkan bahwa model memiliki kinerja yang lebih baik pada data pengujian dibandingkan dengan data pelatihan, dengan kesalahan yang lebih rendah pada data pengujian.

R2 Score: R2 Score pada data pelatihan adalah sekitar 0.0009 dan pada data pengujian adalah sekitar 0.0112. Meskipun nilai-nilai ini positif, mereka sangat rendah, menunjukkan bahwa model tidak cocok dengan data dengan baik dan memiliki kinerja yang buruk dalam menjelaskan variasi dalam data.

MAPE (Mean Absolute Percentage Error): MAPE pada data pelatihan adalah sekitar 325.20 dan pada data pengujian adalah sekitar 325.35. Ini menunjukkan bahwa model memiliki kesalahan relatif yang besar dalam memprediksi data, terutama ketika diukur sebagai persentase dari nilai aktual.

Mean (Rata-rata): Nilai rata-rata target adalah sekitar 9.55. Ini memberikan konteks tentang skala nilai target yang digunakan dalam evaluasi model.

Secara keseluruhan, hasil tes menunjukkan bahwa model yang digunakan belum sepenuhnya sesuai dengan data, dengan kinerja yang lebih baik pada data pengujian dibandingkan dengan data pelatihan, tetapi masih belum memadai dalam menjelaskan variasi dalam data. Diperlukan peninjauan lebih lanjut terhadap model dan fitur yang digunakan untuk perbaikan yang lebih baik.